# Example of using C++ code in python and Jupyter with SWIG

It is also possible to use our C++ code from python and Jupyter. This involves using the [SWIG](http://www.swig.org) package. You can download it [here](http://www.swig.org/survey.html) and then install following instructions [here](http://www.swig.org/Doc3.0/Preface.html#Preface_installation). If you are successful, you should be able to open a new terminal and type ```which swig``` to obtain the path of swig. 

The idea is then to use SWIG to automatically generate python-readable code from our C++/C libraries. There is a lot to learn in this regard, so we will try first with a simple example that illustrates some concepts we will need, such as using STL libraries and C++11 compilation. 



## Step 1 : Look at C++ files

You should be able to see these two simple C++ files: 


In [1]:
! cat swig_example/example.hpp swig_example/example.cpp 

#ifndef EXAMPLE_HPP
#define EXAMPLE_HPP

#include <vector>

// Declare your function normally
int sum_int(std::vector<int> const & vec);

#endif // EXAMPLE_HPP
#include "example.hpp"

// Define your function as you need.
int sum_int(std::vector<int> const & vec)
{
  int sum = 0;
  for ( auto i : vec ) {
    sum += i;
  }
  return sum;
}




## Step 2 : Look at SWIG interface file

The magic of SWIG is to create wrapper C++ functions that use the "cython" interface. You will see an "interface" file for SWIG :

In [2]:
! cat swig_example/example.i 

%module example
/* First: Include your own code.*/
%{
#define SWIG_FILE_WITH_INIT
#include "example.hpp"
%}
/* Next: declare the functions you want to use.*/
int sum_int(std::vector<int> const & vec);

/* Finally: include any other libraries and types you need. 
Yes, you need one type per class, so that means one type per
template instance.*/


%include "std_vector.i"

namespace std {
   %template(vector_int) vector<int>;
};




## Step 3 : Look at SWIG setup file

This tells SWIG to generate a file called ```example_wrap.cc``` using source from ```example.cpp``` with C++11. 


In [3]:
! cat swig_example/setup.py

#!/usr/bin/env python

"""
setup.py file for SWIG example
"""

from distutils.core import setup, Extension


example_module = Extension('_example',
                           sources=['swig_example/example_wrap.cxx', 'swig_example/example.cpp'],
                           extra_compile_args=["-std=c++11"],
                           )

setup (name = 'example',
       version = '0.1',
       author      = "SWIG Docs",
       description = """Simple swig example from docs""",
       ext_modules = [example_module],
       py_modules = ["example"],
       )


Now create the interface using the interface file. This will create two files, ```example_wrap.cxx``` and ```example.py```. They are not intended to be human-readable, so we don't need to look into them. 

## Step 4: Create SWIG interface from file

In [4]:
! swig -c++ -python swig_example/example.i 

## Step 5 : compile C++ and interface

This will compile our file ```example.cpp``` along with ```example_wrap.cxx``` using the rules we set (for instance, using the C++11 compiler). 

We are now free to use this in python!

In [5]:
! python swig_example/setup.py build_ext --inplace

running build_ext
building '_example' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c swig_example/example_wrap.cxx -o build/temp.linux-x86_64-3.7/swig_example/example_wrap.o -std=c++11
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c swig_example/example.cpp -o build/temp.linux-x86_64-3.7/swig_example/example.o -std=c++11
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/swig_example/example_wrap.o build/temp.linux-x86_64-3.7/swig_example/example.o -o /results/CompPhys/SwigExamples/_exam


## Step 6: Make profit: 

First we import the path correctly. 

In [6]:
import sys
import os

sys.path.append( os.path.abspath("swig_example") )

print (sys.path)




['/results/CompPhys/SwigExamples', '/app/compphys_python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/results/.ipython', '/results/CompPhys/SwigExamples/swig_example']


Next we import our actual function from "example.py"

In [7]:
from example import *


And finally, we can use our code: 

In [8]:
a = vector_int([1,2,3])
x = sum_int(a)
print (x)


6


# Success!